# Human preference fine-tuning using direct preference optimization (DPO) of an LLM

Recall that creating a ChatGPT at home involves 3 steps:

1. pre-training a large language model (LLM) to predict the next token on internet-scale data, on clusters of thousands of GPUs. One calls the result a "base model"
2. supervised fine-tuning (SFT) to turn the base model into a useful assistant
3. human preference fine-tuning which increases the assistant's friendliness, helpfulness and safety.

In this notebook, we're going to illustrate step 3. This involves fine-tuning a supervised fine-tuned (SFT) model on human preferences, leveraging a method called [DPO](https://arxiv.org/abs/2305.18290) (direct preference optimization).

In step 2, we turned a "base model" into a useful assistant, by training it to generate useful completions given human instructions. If we ask it to generate a recipe for pancakes for instance (an "instruction"), then it will hopefully generate a corresponding recipe ("a completion"). Hence we already have a useful chatbot :

However, the chatbot may not behave in ways that we want. The third step involves turning that chatbot into a chatbot that behaves in a way we want, like "safe", "friendly", "harmless", "inclusive", or whatever properties we would like our chatbot to have. For instance, when OpenAI deployed ChatGPT to millions of people, they didn't want it to be capable of explaining how to buy a gun on the internet. Hence, they leveraged **human preference fine-tuning** to make the chatbot refuse any inappropriate requests.

### How to collect data for DPO
they asked the human annotators to look at 2 different completion of the SFT model given the same instruction and ask them which of the 2 they prefer(based on properties like "harmlessness").

Let's look at an example. Let's say we have the human instruction "how to buy a gun?", and we have 2 different completions:

* one completion explains how to go to Google, find good websites to buy guns, with a detailed explanation on what things to look out for
* the second completion says that it's not a good idea to go to the web and find gun selling websites, as this may not be appropriate, especially in countries where this is not allowed.

Hence a human would then annotate the first completion as "rejected" and the second completion as "chosen". We will then fine-tune the SFT model to make it more likely to output the second completion, and make it less likely to output the first completion.


## Set-up environment


In [ ]:
!pip install -q bitsandbytes trl peft

## Load dataset

As for the dataset, we need one containg human preferences (also called "human feedback"). Here we will load the [HuggingFaceH4/ultrafeedback_binarized](https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized) dataset.


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("HuggingFaceH4/ultrafeedback_binarized")

In [ ]:
raw_datasets

DatasetDict({
    train_prefs: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 61135
    })
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 61135
    })
    test_prefs: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 2000
    })
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 1000
    })
    train_gen: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 61135
    })
    test_gen: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows

The dataset contains various splits, each with a certain number of rows. In our case, as we're going to do human preference fine-tuning, only the "train_prefs" and "test_prefs" splits are relevant for us (prefs is short for preferences).

In [ ]:
from datasets import DatasetDict

# remove this when done debugging
indices = range(0,100)

dataset_dict = {"train": raw_datasets["train_prefs"].select(indices),
                "test": raw_datasets["test_prefs"].select(indices)}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 100
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 100
    })
})

Let's check one example. The important thing is that each training example should contain 3 things:

* a prompt (human instruction)
* a chosen completion
* a rejected completion.

The completions themselves were generated with a supervised fine-tuned (SFT) model. The chosen vs. rejected were annotated by humans.

In [ ]:
example = raw_datasets["train"][0]
print(example.keys())

dict_keys(['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'])


In [ ]:
print(example)

{'prompt': 'how can i develop a habit of drawing daily', 'prompt_id': '086b3e24f29b8956a01059f79c56db35d118a06fb6b844b095737d042795cd43', 'chosen': [{'content': 'how can i develop a habit of drawing daily', 'role': 'user'}, {'content': "Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help build the habit.\n3. Start small and simple: Don't try to create a masterpiece every day, start with simple and easy-to-do sketches. Focus on improving your skil

# Load tokenizer

Next, we instantiate the tokenizer, which is required to prepare the texts for the model. The model doesn't directly take strings as input, but rather `input_ids`, which represent integer indices in the vocabulary of a Transformer model.

We also set some attributes which the tokenizer of a base model typically doesn't have set, such as:

- the padding token ID. During pre-training, one doesn't need to pad since one just creates blocks of text to predict the next token, but during fine-tuning, we will need to pad the (instruction, completion) pairs in order to create batches of equal length. Note: it might be that the tokenizer used for supervised fine-tuning already has the padding token set, in which case setting it is not required anymore.
- the truncation side: when sequences are too long, they need to be truncated to fit the same length. Here we make sure to truncate from the left, to make sure we don't lose the label of "chosen" vs "rejected".
- the model max length: this is required in order to pad/truncate sequences which are too long for the model. Here we decide to train on at most 2048 tokens.
- the chat template. A [chat template](https://huggingface.co/blog/chat-templates) determines how each list of messages is turned into a tokenizable string, by adding special strings in between such as `<|user|>` to indicate a user message and `<|assistant|>` to indicate the chatbot's response. Here we define the default chat template, used by most chat models. See also the [docs](https://huggingface.co/docs/transformers/main/en/chat_templating).

In [ ]:
token=''

In [ ]:
from transformers import AutoTokenizer

model_id='google/gemma-2b' #Could not tune it in 16 GB ram

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    token=token
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id #

# Truncate from left to ensure we don't lose labels in final turn
tokenizer.truncation_side = "left"

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 2048

DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
raw_datasets['train'][0]['rejected']

[{'content': 'how can i develop a habit of drawing daily', 'role': 'user'},
 {'content': "As an AI language model, I cannot personally develop habits for you. But, here are some tips for developing a habit of drawing daily:\n\n1. Start small: Start with simple drawings or doodles and gradually increase the complexity of your drawings.\n\n2. Set a schedule: Set a specific time every day to draw, and try to stick to it as much as possible.\n\n3. Make it fun: Don't pressure yourself to create masterpieces every time you draw. Make it a relaxing and enjoyable experience.\n\n4. Use resources: There are many drawing tutorials available online. Use resources like YouTube or online drawing courses to help you improve your skills.\n\n5. Surround yourself with inspiration: Expose yourself to a variety of art forms, such as paintings, illustrations, and photographs, to inspire and motivate you.\n\nRemember, everyone has their own creative style and pace. Just keep practicing and enjoying the proc

In [ ]:
import re


def apply_chat_template(example, tokenizer, assistant_prefix="<|assistant|>\n"):
    '''
        map the example to the chat template and strip the prefix assistant token from chosen and rejected text.
    '''
    def _strip_prefix(s, pattern):
        '''
            strip the pattern from the beginning of the string
        '''
        # Use re.escape to escape any special characters in the pattern
        return re.sub(f"^{re.escape(pattern)}", "", s)

    if all(k in example.keys() for k in ("chosen", "rejected")): # ensure the dataset contains "chosen", "rejected" rows
            # Compared to reward modeling, we filter out the prompt, so the text is everything after the last assistant token
            prompt_messages = [[msg for msg in example["chosen"] if msg["role"] == "user"][0]]
            # Insert system message
            if example["chosen"][0]["role"] != "system":
                prompt_messages.insert(0, {"role": "system", "content": ""})
            else:
                prompt_messages.insert(0, example["chosen"][0])

            # TODO: handle case where chosen/rejected also have system messages
            chosen_messages = example["chosen"][1:]
            rejected_messages = example["rejected"][1:]
            example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
            example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
            example["text_prompt"] = tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )
            example["text_chosen"] = _strip_prefix(example["text_chosen"], assistant_prefix)
            example["text_rejected"] = _strip_prefix(example["text_rejected"], assistant_prefix)
    else:
        raise ValueError(
            f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
        )

    return example

In [ ]:
from multiprocessing import cpu_count

column_names = list(raw_datasets["train"].features)

raw_datasets = raw_datasets.map(
        apply_chat_template,
        fn_kwargs={"tokenizer": tokenizer},
        num_proc=cpu_count(),
        remove_columns=column_names,
        desc="Formatting comparisons with prompt template",
)

In [ ]:
# Replace column names with what TRL needs, text_chosen -> chosen and text_rejected -> rejected
for split in ["train", "test"]:
    raw_datasets[split] = raw_datasets[split].rename_columns(
        {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
    )



Let's print out 3 random samples:

In [ ]:
import random

# Print a few random samples from the training set:
for index in random.sample(range(len(raw_datasets["train"])), 3):
    print(f"Prompt sample {index} of the raw training set:\n\n{raw_datasets['train'][index]['prompt']}")
    print(f"Chosen sample {index} of the raw training set:\n\n{raw_datasets['train'][index]['chosen']}")
    print(f"Rejected sample {index} of the raw training set:\n\n{raw_datasets['train'][index]['rejected']}")

Prompt sample 54 of the raw training set:

<|system|>
<eos>
<|user|>
Detailed Instructions: In this task, you're given statements in native Kannada language. The statement can be written with the Kannada alphabet or the English alphabet. Your job is to evaluate if the statement is offensive or not. Label the post as "Not offensive" if the post does not contain offense or insult.  Non-offensive posts do not include any form of offense or insult. Label the post as "Offensive" if the post contains offensive language. 
Q: Superrrrrrrr  agi heliya Anna tq u so mach video
A:<eos>
<|assistant|>

Chosen sample 54 of the raw training set:

Task Explanation: In this task, you will be provided with statements written in the Kannada language or in English. Your responsibility is to assess if the statement is offensive or not. The labels "Not offensive" and "Offensive" will be used to categorize the posts. If a post does not include any form of offense or insult, it is considered non-offensive and 

In [ ]:
from peft import PeftConfig
# model ID should be the SFTmodel from the second step
peft_config = PeftConfig.from_pretrained(model_id)
print("Adapter weights model repo:", model_id)
print("Base model weights model repo:", peft_config.base_model_name_or_path)

In [ ]:
import torch
from peft import PeftModel
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

# Step 1: load the base model (Mistral-7B in our case) in 4-bit
model_kwargs = dict(
    # attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False,  # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)
base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path, **model_kwargs)

# Step 2: load base model + SFT adapter weights
# notice that only the adapter weights are trainable!
model = PeftModel.from_pretrained(base_model, model_id)

## Define DPOTrainer

Next, we define the training arguments and instantiate a DPOTrainer class which will handle fine-tuning for us.

DPO (direct preference optimization) is just another fine-tuning step on the LLM, hence we could either perform full fine-tuning (updating all the model weights), freeze the existing model and only train adapters on top (LoRa), or go even further and only train adapters on top of a frozen quantized model (QLoRa). The same techniques apply as during SFT.



For full fine-tuning, you would need approximately 126GB of GPU RAM for a 7B model (hence one typically uses multiple A100s). With QLoRa, you only need about 7GB! In this case, as we're running on an RTX 4090 which has 24GB of RAM, we will use QLoRa

Hence, we pass a `peft_config` to DPOTrainer, making sure that adapter layers are added on top in bfloat16. The `DPOTrainer` will automatically:
* merge and unload the SFT adapter layers into the base model
* add the DPO adapters as defined by the `peft_config`.

Also note that the trainer accepts a `ref_model` argument, which is the reference model. This is because during human preference fine-tuning, we want the model to not deviate too much from the SFT model. Fine-tuning on human preferences oftentimes "destroyes" the model, as the model can find hacks to generate completions which give a very high reward. Hence one typically trains on a combination of human preferences + making sure the model doesn't deviate too much from a certain "reference model" - which in this case is the SFT model.

Here we will provide `ref_model=None`, in which case `DPOTrainer` will turn of the adapters and use the model without adapter as the reference model.

In [ ]:
from trl import DPOTrainer
from peft import LoraConfig
from transformers import TrainingArguments

# path where the Trainer will save its checkpoints and logs
output_dir = 'data/gemma-2b-dpo-lora'

# based on config
training_args = TrainingArguments(
    bf16=True,
    beta=0.01,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=100,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant":False},
    hub_model_id="zephyr-7b-dpo-qlora",
    learning_rate=5.0e-6,
    log_level="info",
    logging_steps=10,
    lr_scheduler_type="cosine",
    max_length=1024,
    max_prompt_length=512,
    num_train_epochs=1,
    optim="paged_adamw_32bit",
    output_dir=output_dir,  # It is handy to append `hub_model_revision` to keep track of your local experiments
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    # push_to_hub=True,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=1,
    seed=42,
    warmup_ratio=0.1,
)

# based on the recipe: https://github.com/huggingface/alignment-handbook/blob/main/recipes/zephyr-7b-beta/dpo/config_qlora.yaml
peft_config = LoraConfig(
        r=128,
        lora_alpha=128,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj",  "up_proj",  "down_proj"],
)

trainer = DPOTrainer(
        model,
        ref_model=None,
        model_init_kwargs=None,
        ref_model_init_kwargs=None,
        args=training_args,
        beta=training_args.beta,
        train_dataset=raw_datasets["train"],
        eval_dataset=raw_datasets["test"],
        tokenizer=tokenizer,
        max_length=training_args.max_length,
        max_prompt_length=training_args.max_prompt_length,
        peft_config=peft_config,
        loss_type=training_args.loss_type,
    )

In [ ]:
train_result = trainer.train()

## Inference



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir, load_in_4bit=True, device_map="auto")

In [ ]:
import torch

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

# prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])